This file will create a feature that divides comment upvotes by article upvotes to ensure that comments on articles with more impressions do not seem better to the model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

import os
from dotenv import load_dotenv
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET=os.getenv('CLIENT_SECRET')
APP_NAME=os.getenv('APP_NAME')
REDDIT_USERNAME=os.getenv('REDDIT_USERNAME')
REDDIT_PASSWORD=os.getenv('REDDIT_PASSWORD')

import praw
import pandas as pd
import datetime as dt

reddit = praw.Reddit(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, user_agent=APP_NAME, username=REDDIT_USERNAME, password=REDDIT_PASSWORD)

print(reddit.user.me())

%matplotlib inline

Version 7.1.0 of praw is outdated. Version 7.1.3 was released 1 day ago.


mattcat26


In [23]:
comments_arr = []
subreddit = reddit.subreddit('neutralnews')

for post in subreddit.new(limit = 10):
    submission = reddit.submission(id=post.id)
    submission.comments.replace_more(limit=100)
    for comment in submission.comments:
        comments_arr.append([post.title, post.score, comment.score, post.url, comment.body, comment.author, post.id, comment.id])

comment_data = pd.DataFrame(comments_arr, columns=['post_title', 'article_score', 'comment_score', 'url', 'content', 'author', 'submissionId', 'commentId'])

In [24]:
comment_data

,post_title,article_score,comment_score,url,content,author,submissionId,commentId
0,Judge Orders OAN to Pay Rachel Maddow and MSNB...,154,1,https://lawandcrime.com/first-amendment/judge-...,"r/NeutralNews is a **curated space**, but desp...",NeutralverseBot,lempbp,gmfm93c
1,Judge Orders OAN to Pay Rachel Maddow and MSNB...,154,1,https://lawandcrime.com/first-amendment/judge-...,[removed],None,lempbp,gmfqaru
2,Judge Orders OAN to Pay Rachel Maddow and MSNB...,154,1,https://lawandcrime.com/first-amendment/judge-...,__I'm a bot.__\n\nThe [linked_article](https:/...,TheFactualBot,lempbp,gmfmctq
3,What of 'Individual-1'? Feds' Trump campaign c...,58,1,https://apnews.com/article/donald-trump-campai...,"r/NeutralNews is a **curated space**, but desp...",NeutralverseBot,le3xhg,gm998oz
4,What of 'Individual-1'? Feds' Trump campaign c...,58,1,https://apnews.com/article/donald-trump-campai...,__I'm a bot. Here are The Factual credibility ...,TheFactualBot,le3xhg,gm9a5av
5,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,"r/NeutralNews is a **curated space**, but desp...",NeutralverseBot,le0bp7,gm8o5oy
6,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,[The full PDF is quite readable.](https://www....,rickosborne,le0bp7,gm8tvmd
7,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,[removed],None,le0bp7,gm9fy42
8,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,I largely agree with this outcome.\n\nGovernme...,Kerrizma,le0bp7,gmc04vt
9,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,This is so fucked. USA is crumbling because of...,Fyrbyk,le0bp7,gmf50oe


In [25]:
new_arr = comment_data['post_title'].unique()
for element in new_arr:
    print(element + '\n')

Judge Orders OAN to Pay Rachel Maddow and MSNBC $250,000 in Attorney Fees For Filing Frivolous Defamation Lawsuit

What of 'Individual-1'? Feds' Trump campaign case is 'dead'

Justices: California can't enforce indoor church service ban

Analysis: A race war evident long before the Capitol siege

The Secret Bipartisan Campaign That Saved the 2020 Election

The Left’s Vaccine Problem

Klobuchar targets Big Tech with biggest antitrust overhaul in 45 years

Nevada bill would allow tech companies to create governments

Lawyers Call Trump's First Amendment Defense 'Legally Frivolous'

Pentagon, stumped by extremism in ranks, orders stand-down in next 60 days



In [26]:
comment_data['score_ratio'] = comment_data['comment_score'] / comment_data['article_score']
comment_data

,post_title,article_score,comment_score,url,content,author,submissionId,commentId,score_ratio
0,Judge Orders OAN to Pay Rachel Maddow and MSNB...,154,1,https://lawandcrime.com/first-amendment/judge-...,"r/NeutralNews is a **curated space**, but desp...",NeutralverseBot,lempbp,gmfm93c,0.006494
1,Judge Orders OAN to Pay Rachel Maddow and MSNB...,154,1,https://lawandcrime.com/first-amendment/judge-...,[removed],None,lempbp,gmfqaru,0.006494
2,Judge Orders OAN to Pay Rachel Maddow and MSNB...,154,1,https://lawandcrime.com/first-amendment/judge-...,__I'm a bot.__\n\nThe [linked_article](https:/...,TheFactualBot,lempbp,gmfmctq,0.006494
3,What of 'Individual-1'? Feds' Trump campaign c...,58,1,https://apnews.com/article/donald-trump-campai...,"r/NeutralNews is a **curated space**, but desp...",NeutralverseBot,le3xhg,gm998oz,0.017241
4,What of 'Individual-1'? Feds' Trump campaign c...,58,1,https://apnews.com/article/donald-trump-campai...,__I'm a bot. Here are The Factual credibility ...,TheFactualBot,le3xhg,gm9a5av,0.017241
5,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,"r/NeutralNews is a **curated space**, but desp...",NeutralverseBot,le0bp7,gm8o5oy,0.004608
6,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,[The full PDF is quite readable.](https://www....,rickosborne,le0bp7,gm8tvmd,0.004608
7,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,[removed],None,le0bp7,gm9fy42,0.004608
8,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,I largely agree with this outcome.\n\nGovernme...,Kerrizma,le0bp7,gmc04vt,0.004608
9,Justices: California can't enforce indoor chur...,217,1,https://apnews.com/article/politics-us-supreme...,This is so fucked. USA is crumbling because of...,Fyrbyk,le0bp7,gmf50oe,0.004608


Something to note, if the article score is zero, the score ratio will be inf. If the article score and the comment score is zero, the score ratio will be NaN. If the comment score is zero, the score ratio will be zero.